<a href="https://colab.research.google.com/github/padmacharan-123/LSTM/blob/main/lstm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow scikit-learn pandas openpyxl imbalanced-learn --quiet

import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("synthetic_sensor_dataset.csv")

print("\n Dataset loaded successfully!")
print(df.head())
required_cols = ["voltage_rms", "current_rms", "power_factor", "active_power", "apparent_power", "fault_type"]
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")
X = df[["voltage_rms", "current_rms", "power_factor", "active_power", "apparent_power"]].values
y = df["fault_type"].values
label_encoder = LabelEncoder()
y_int = label_encoder.fit_transform(y)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_int)
y_onehot = tf.keras.utils.to_categorical(y_resampled)

print("\n Classes:", label_encoder.classes_)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=42)
print("\n Train shape:", X_train.shape, "Test shape:", X_test.shape)
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(y_onehot.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=1)
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print("\n Classification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_, zero_division=0))
def ml_to_alert(pred_idx, sample):
    severity_map = {
        "Normal": ("low", "Normal operation"),
        "Sag": ("medium", "Voltage sag detected"),
        "Short Circuit": ("high", "Short circuit detected"),
        "Outage": ("high", "Power outage detected"),
        "Wire Cut": ("high", "Wire cut detected"),
        "PoleFall": ("high", "Pole fall detected")
    }
    label = label_encoder.inverse_transform([pred_idx])[0]
    severity, message = severity_map.get(label, ("unknown", "Unknown fault"))

    return {
        "fault_type": label,
        "severity": severity,
        "message": message,
        "features": sample.flatten().tolist(),
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
idx = np.random.randint(0, X_test.shape[0])
pred = np.argmax(model.predict(X_test[idx:idx+1]), axis=1)[0]
alert = ml_to_alert(pred, X_test[idx])
print("\n --- New Event Detection ---")
print(alert)


Saving synthetic_sensor_dataset.csv to synthetic_sensor_dataset (1).csv

✅ Dataset loaded successfully!
             timestamp  voltage_rms  current_rms  power_factor  active_power  \
0  2025-01-01 00:00:00   232.492246     9.033360      0.948965   1993.003260   
1  2025-01-01 00:01:00    30.402812   114.437787      0.338557   1177.919565   
2  2025-01-01 00:02:00     0.483698    -0.070018      0.000000     -0.000000   
3  2025-01-01 00:03:00    64.807002    70.095284      0.253477   1151.462759   
4  2025-01-01 00:04:00     0.215729     0.004828      0.000000      0.000000   

   apparent_power     fault_type  
0     2100.186056         Normal  
1     3479.230474  Short Circuit  
2       -0.033868         Outage  
3     4542.665231  Short Circuit  
4        0.001042       Wire Cut  

 Classes: ['Normal' 'Outage' 'Sag' 'Short Circuit' 'Wire Cut']

 Train shape: (852, 1, 5) Test shape: (213, 1, 5)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,501 (119.14 KB)

 Trainable params: 30,501 (119.14 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4479 - loss: 1.5870
Epoch 2/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6570 - loss: 1.4669
Epoch 3/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6623 - loss: 1.2118
Epoch 4/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6559 - loss: 0.8389
Epoch 5/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7518 - loss: 0.5936
Epoch 6/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8139 - loss: 0.4600
Epoch 7/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.3893
Epoch 8/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7908 - loss: 0.3574
Epoch 9/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8082 - loss: 0.3086
Epoch 10/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7912 - loss: 0.3172
Epoch 11/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7732 - loss: 0.3138
Epoch 12/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8014 - lo